# Recommendation with regression model TPH

In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
#import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
import pickle
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [ ]:
tag_select=['min_water_3',# 1
 'min_solid percentage_10',# 2
 'HH TPH',# 3
 'max_delta LL charge cell_10',# 4
 'power',# 5
 'rms_delta LL charge cell_3',# 6
 'var_delta HH charge cell_10',# 7
 'max_covelin law_10',# 8
 'LL charge cell_(t-2)',# 9
 'min_granulometry_5',# 10
 'max_bornite law_10', # 11
 'min_charge cell_5',# 12
 'chalcocite law_(t-2)',# 13
 'max_sag power index_5',# 14
 'min_speed_3',# 15
 'var_bornite law_3',# 16
 'var_speed_3',# 17
 'min_pyrite law_10', # 18
 'crusher index_(t-5)', # 19
 'var_power_3',# 20
 'var_chalcocite law_3',# 21
 'var_bornite law_5',# 22
 'var_solid percentage_3',# 23
 'var_speed_10',# 24
 'ball work index_(t-1)',# 25
 'var_chalcocite law_5', # 26
 'var_water_3',# 27
 'chalcopyrite law_(t-5)',# 28
 'var_crusher index_10',# 29
 'var_chalcopyrite law_3',# 30
 'var_granulometry_3',# 31
 'var_delta HH charge cell_3'# 32}
 ]

In [ ]:
def pipeline_feature_engineering(df):
      
    df_vars = df.copy()
    
    pipe_df_features=pd.DataFrame()

    pipe_df_features["delta HH charge cell"]=df_vars["HH charge cell"]-df_vars["charge cell"]
    pipe_df_features["delta LL charge cell"]=df_vars["charge cell"]-df_vars["LL charge cell"]
    pipe_df_features["TPH"]=df_vars["charge cell"]-df_vars["LL charge cell"]
    pipe_df_features['HH charge cell']=df_vars["charge cell"]
    pipe_df_features['LL charge cell']=df_vars["charge cell"]
    pipe_df_features['charge cell']=df_vars["charge cell"]

    pipe_df_features["min_water_3"]=df_vars["water"].rolling(window=3).apply(lambda x:np.nanmin(list(x))) # 1
    pipe_df_features["min_solid percentage_10"]=df_vars["solid percentage"].rolling(window=10).apply(lambda x:np.nanmin(list(x))) # 2
    pipe_df_features['HH TPH']=df_vars['HH TPH'] # 3
    pipe_df_features["max_delta LL charge cell_10"]=pipe_df_features["delta LL charge cell"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 4
    pipe_df_features['power']=df_vars['power'] # 5
    pipe_df_features["rms_delta LL charge cell_3"]=pipe_df_features["delta LL charge cell"].rolling(window=3).apply(lambda x: sqrt(mean(square(list(x))))) # 6
    pipe_df_features["var_delta HH charge cell_10"]=pipe_df_features["delta HH charge cell"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 7
    pipe_df_features[f"max_covelin law_10"]=df_vars["covelin law"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 8
    pipe_df_features[f'LL charge cell_(t-2)'] = pipe_df_features["LL charge cell"].shift(2) # 9
    pipe_df_features[f"min_granulometry_5"]=df_vars["granulometry"].rolling(window=5).apply(lambda x:np.nanmin(list(x))) # 10
    pipe_df_features[f"max_bornite law_10"]=df_vars["bornite law"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 11
    pipe_df_features[f"min_charge cell_5"]=pipe_df_features["charge cell"].rolling(window=5).apply(lambda x:np.nanmin(list(x))) # 12
    pipe_df_features[f'chalcocite law_(t-2)'] = df_vars["chalcocite law"].shift(2) # 13
    pipe_df_features[f"max_sag power index_5"]=df_vars["sag power index"].rolling(window=5).apply(lambda x:np.nanmax(list(x))) # 14
    pipe_df_features[f"min_speed_3"]=df_vars["speed"].rolling(window=3).apply(lambda x:np.nanmin(list(x))) # 15
    pipe_df_features[f"var_bornite law_3"]=df_vars["bornite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 16
    pipe_df_features[f"var_speed_3"]=df_vars["speed"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 17
    pipe_df_features[f"min_pyrite law_10"]=df_vars["pyrite law"].rolling(window=10).apply(lambda x:np.nanmin(list(x))) # 18
    pipe_df_features[f'crusher index_(t-5)'] = df_vars["crusher index"].shift(5) # 19
    pipe_df_features[f"var_power_3"]=df_vars["power"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 20
    pipe_df_features[f"var_chalcocite law_3"]=df_vars["chalcocite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 21
    pipe_df_features[f"var_bornite law_5"]=df_vars["bornite law"].rolling(window=5).apply(lambda x: np.nanvar(x)) # 22
    pipe_df_features[f"var_solid percentage_3"]=df_vars["solid percentage"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 23
    pipe_df_features[f"var_speed_10"]=df_vars["speed"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 24
    pipe_df_features[f"ball work index_(t-1)"] = df_vars["ball work index"].shift(1) # 25
    pipe_df_features[f"var_chalcocite law_5"]=df_vars["chalcocite law"].rolling(window=5).apply(lambda x: np.nanvar(x)) # 26
    pipe_df_features[f"var_water_3"]=df_vars["water"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 27
    pipe_df_features[f'chalcopyrite law_(t-5)'] = df_vars["chalcopyrite law"].shift(5) # 28
    pipe_df_features[f"var_crusher index_10"]=df_vars["crusher index"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 29
    pipe_df_features[f"var_chalcopyrite law_3"]=df_vars["chalcopyrite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 30
    pipe_df_features[f"var_granulometry_3"]=df_vars["granulometry"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 31
    pipe_df_features[f"var_delta HH charge cell_3"]=pipe_df_features["delta HH charge cell"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 32
    
    return pipe_df_features

In [ ]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'speed','HH TPH', 'granulometry','Edad','loss of TPH',"solid percentage","HH charge cell","LL charge cell",
       "recommendation base model carl","recommendation base model acn","water","power"]

In [ ]:
# read df full csv
df_features0=pd.read_csv('../../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#["2020":"2021"]
df_features0["TPH"]=df_features0["TPH"].shift(-10)
df_features0["loss of TPH"]=df_features0["loss of TPH"].shift(-10)
df_features0.dropna(inplace=True)
df_features0.sort_index(inplace=True)
df_features0.shape

In [ ]:
# Verificando si hay duplicados
u=[i for i in df_features0.columns if not re.match(".*delta HH TPH",i) ]
df_features=df_features0[u].copy()
df_features.head(1)

In [ ]:
df_features["HH charge cell"].describe()

In [ ]:
df_features["HH charge cell"].plot()

# Load pipeline and XGBoost

In [ ]:
# Load
model= xgb.XGBRegressor()
model.load_model("../../../models/model_xgb.bin")

In [ ]:
# Leer pipeline:
pipe= pickle.load(open('../../../models/pipe_xgb.pkl', 'rb'))

# Recommendation base model ACN

In [ ]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//..//models//pickles//Modelo update Accenture//cc_recommendations_prod.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK['s'+subcontexto].loc[granulometria]["cc"]
  
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [ ]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [ ]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model acn"]=Rec_HH_CC

# Recommendation base model Carl

In [ ]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//..//models//pickles//Curvas de celda de carga vs granulometria productivo actualizado.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK[subcontexto].loc[granulometria]["cc"]
    
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [ ]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [ ]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model carl"]=Rec_HH_CC

# Data test para recomendación HH CC

In [ ]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
test.head(1)

In [ ]:
# Ytest reg
Ytest=test["TPH"]
Xtest=test.drop(columns=["TPH"])

# Simulation time series predict

# 2020

In [ ]:
# test 2020
test_time_serie=Xtest["2020"]
test_time_serie["TPH"]=Ytest["2020"]
Xtest1=pipe.transform(test_time_serie)
Ypred= model.predict(Xtest1)
test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# 2021

In [ ]:
# test 2020
test_time_serie=Xtest["2021"]
test_time_serie["TPH"]=Ytest["2021"]
Xtest1=pipe.transform(test_time_serie)
Ypred= model.predict(Xtest1)
test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=7, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['water'], ##FF6511
                    mode='lines',
                    name='water',line=dict(width=3, ),legendgroup = '1'),row=7, col=1)  
#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# 2022

In [ ]:
# test 2020
test_time_serie=Xtest["2022"]
test_time_serie["TPH"]=Ytest["2022"]
Xtest1=pipe.transform(test_time_serie)
Ypred= model.predict(Xtest1)
test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# recommendation

In [ ]:
inicio_periodo=parse('2021-09-15 03:13:00')
list_dfs=[]
list_fechas=[]
for i in range(1,33):
    print(i)
    inicio_periodo=inicio_periodo+timedelta(minutes=1)
    fin_periodo=inicio_periodo+timedelta(minutes=10)
    print(inicio_periodo)
    print(fin_periodo)
    print("")

    list_TPH=[]
    list_HH_CC=[]

    # Proceso de optimización
    for i in np.arange(650,1050,5):

        list_HH_CC.append(i)
        
        # Input de función a optimizar
        data=test_time_serie.copy()[inicio_periodo:fin_periodo][variables_interes+["TPH"]]
        data["HH charge cell"]=i

        # función de optmización
        data["LL charge cell"]=i-80
        data["charge cell"]=(data["HH charge cell"]+data["LL charge cell"])/2
        data=pipeline_feature_engineering(data)[tag_select+["TPH","HH charge cell","LL charge cell","charge cell"]]
        
        X=pipe.transform(data)
        Ypred= model.predict(X)
        data["TPH predict"]=Ypred
        # función de optimización

        # Output función
        list_TPH.append(Ypred[-1])
        
        #display(data.tail())
        #print(i)

    df_tph=pd.DataFrame({"TPH":list_TPH,"HH CC":list_HH_CC})
    
    list_dfs.append(df_tph)
    list_fechas.append(fin_periodo)

    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_tph["HH CC"], y=df_tph["TPH"],
                        mode='lines',
                        name='TPH'))


    fig.update_layout(height=500, width=1200, title_text="TPH vs HH charge cell" ,xaxis_title="HH charge cell",
        yaxis_title="TPH")   

    fig.update_layout(hovermode="x unified")                
    fig.show()

In [ ]:
HH_TPH_event=test_time_serie.loc['2021-09-15 03:17:00']["HH TPH"]
HH_TPH_event

In [ ]:
list_opt_tph=[]
list_opt_hh=[]
for frames in list_dfs:
    x=frames.copy()
    # optimo
    opt=x["TPH"].max()
    optHH=x[x["TPH"]==opt].iloc[-1]["HH CC"]
    list_opt_tph.append(opt)
    list_opt_hh.append(optHH)
    print(optHH,opt)

df_opt=pd.DataFrame({"Timestamp":list_fechas,"TPH predict opt":list_opt_tph,"recommended HH charge cell":list_opt_hh})
df_opt["HH TPH"]=HH_TPH_event
df_opt.set_index("Timestamp",inplace=True)
df_opt=df_opt.join(test_time_serie[["TPH","TPH predict","charge cell","LL charge cell","HH charge cell",'recommendation base model carl','recommendation base model acn']])
df_opt

In [ ]:
# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["TPH predict opt"],
                    mode='lines',
                    name='TPH predict opt'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["TPH predict"],
                    mode='lines',
                    name='TPH predict'))
fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["HH TPH"],
                    mode='lines',
                    name='HH TPH'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["TPH"],
                    mode='lines',
                    name='TPH'))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict & TPH predict opt",yaxis_title="TPH"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["HH charge cell"],
                    mode='lines',
                    name='HH charge cell'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt['recommendation base model acn'],
                    mode='lines',
                    name='recommendation base model acn'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt['recommendation base model carl'],
                    mode='lines',
                    name='recommendation base model carl'))


fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["recommended HH charge cell"],
                    mode='lines',
                    name='recommended HH charge cell ML model'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["recommended HH charge cell"]-80,
                    mode='lines',
                    name='recommended LL charge cell ML model'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["LL charge cell"],
                    mode='lines',
                    name='LL charge cell'))

fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["charge cell"],
                    mode='lines',
                    name='charge cell'))                   

fig.update_layout(height=500, width=1200, title_text="recommended HH charge cell vs HH charge cell",yaxis_title="charge cell"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()